In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from operations import adjust_dataset, split_data, Ann




2023-12-04 08:22:52.077621: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 08:22:52.079477: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 08:22:52.112250: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 08:22:52.112326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 08:22:52.113255: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# Implementation of ANN

In [2]:

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Abu Dhabi')



In [3]:
possibilities = ['temperature_celsius', 'wind_kph', 'humidity', 'pressure_mb', 'precip_mm', 'cloud']

target = 'temperature_celsius'
X_train, y_train, X_test, y_test = split_data(df, target, int(len(df) * 0.7))



In [6]:
model = Ann(y_train, y_test, 7, 3, target)
model.build_model()

Epoch 1/100
1/1 [==============================] - 1s 520ms/step - loss: 832.2211 - mean_squared_error: 832.2211
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 823.3517 - mean_squared_error: 823.3517
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 767.8315 - mean_squared_error: 767.8315
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: 732.5865 - mean_squared_error: 732.5865
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 740.4904 - mean_squared_error: 740.4904
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 709.0562 - mean_squared_error: 709.0562
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 644.8845 - mean_squared_error: 644.8845
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 625.6816 - mean_squared_error: 625.6816
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 630.3026 - mean_squared_error: 630.3

In [7]:
res = model.evaluate()

print(res)

2/2 - 0s - loss: 7.9696 - mean_squared_error: 7.9696 - 102ms/epoch - 51ms/step
2.82


In [8]:
pred = model.predict()

print (pred)

1/1 [==============================] - 0s 83ms/step
[27.659151 27.118246 25.48907 ]


# Investigating the use of random forests for feature extraction

In [9]:
# Lets run the model 10 times for temperature_celsius in Prague

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Prague')

target = 'temperature_celsius'
X_train, y_train, X_test, y_test = split_data(df, target, int(len(df) * 0.7))

results_without_rf = []
for i in range(10):
    model = Ann(y_train, y_test, 7, 3, target)
    model.build_model()
    res = model.evaluate()
    results_without_rf.append(res)
    

Epoch 1/100
1/1 [==============================] - 1s 589ms/step - loss: 27.4506 - mean_squared_error: 27.4506
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 26.6914 - mean_squared_error: 26.6914
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 25.8887 - mean_squared_error: 25.8887
Epoch 4/100
1/1 [==============================] - 0s 13ms/step - loss: 25.4339 - mean_squared_error: 25.4339
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 24.3791 - mean_squared_error: 24.3791
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 22.9719 - mean_squared_error: 22.9719
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 21.7780 - mean_squared_error: 21.7780
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 20.6542 - mean_squared_error: 20.6542
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: 20.8167 - mean_squared_error: 20.8167
Epoch 10/100

In [10]:
# Time execution = 47.5 seconds

print (sum(results_without_rf)/len(results_without_rf))

5.6259999999999994


In [21]:
# Lets apply Random Forest

from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

feature_importances = model.feature_importances_


threshold = 0.001  # threshold mean importance
selected_features = [feature for feature, importance in sorted(enumerate(feature_importances), key=lambda x: x[1], reverse=True) if importance > threshold]

X_selected_train = X_train.iloc[:, selected_features]



print (X_selected_train.columns)

Index(['feels_like_celsius', 'moonrise', 'moon_illumination',
       'moon_phase_Waxing Crescent', 'air_quality_PM2.5', 'humidity',
       'air_quality_Nitrogen_dioxide', 'air_quality_Ozone', 'gust_kph',
       'sunset', 'pressure_mb', 'sunrise', 'moonset', 'wind_kph',
       'air_quality_Sulphur_dioxide'],
      dtype='object')


In [22]:
df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Prague')

target = 'temperature_celsius'
X_train, y_train, X_test, y_test = split_data(df, target, int(len(df) * 0.7))

X_train = X_train.iloc[:, selected_features]
X_test = X_test.iloc[:, selected_features]

results_with_rf = []
for i in range(10):
    model = Ann(y_train, y_test, 7, 3, target)
    model.build_model()
    res = model.evaluate()
    results_with_rf.append(res)


Epoch 1/100
1/1 [==============================] - 1s 584ms/step - loss: 35.1094 - mean_squared_error: 35.1094
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 33.3899 - mean_squared_error: 33.3899
Epoch 3/100
1/1 [==============================] - 0s 7ms/step - loss: 33.8025 - mean_squared_error: 33.8025
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 29.4890 - mean_squared_error: 29.4890
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 29.1898 - mean_squared_error: 29.1898
Epoch 6/100
1/1 [==============================] - 0s 11ms/step - loss: 28.6204 - mean_squared_error: 28.6204
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 25.8284 - mean_squared_error: 25.8284
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 25.6165 - mean_squared_error: 25.6165
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 24.0380 - mean_squared_error: 24.0380
Epoch 10/100


In [23]:
print (sum(results_with_rf)/len(results_with_rf))

5.708


In [ ]:
# CONCLUSION: Random Forest does not improve the results